# Data Cleaning

In [1]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from statsmodels.tsa.stattools import adfuller #to check unit root in time series 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

import seaborn as sns #for correlation heatmap

import warnings
warnings.filterwarnings('ignore')

In [3]:
bigmacrodata=pd.read_csv("data/Macroeconomic_Variables.csv")
bigmacrodata=bigmacrodata.rename(columns={'sasdate':'Date'})
Recession_periods=pd.read_csv('data/Recession_Periods.csv')
bigmacrodata.insert(loc=1,column="Regime", value=Recession_periods['Regime'].values)
bigmacrodata.head()

,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
0,1/1/1959,Normal,2583.560,2426.0,15.188,276676.8154,18235.77392,21.9616,23.3868,22.2620,...,18.294,10.152,2.13,2.45,2.04,NaN,6476.0,12298.0,84.2043,NaN
1,2/1/1959,Normal,2593.596,2434.8,15.346,278713.9773,18369.56308,22.3917,23.7024,22.4549,...,18.302,10.167,2.14,2.46,2.05,NaN,6476.0,12298.0,83.5280,NaN
2,3/1/1959,Normal,2610.396,2452.7,15.491,277775.2539,18523.05762,22.7142,23.8459,22.5651,...,18.289,10.185,2.15,2.45,2.07,NaN,6508.0,12349.0,81.6405,NaN
3,4/1/1959,Normal,2627.446,2470.0,15.435,283362.7075,18534.46600,23.1981,24.1903,22.8957,...,18.300,10.221,2.16,2.47,2.08,NaN,6620.0,12484.0,81.8099,NaN
4,5/1/1959,Normal,2642.720,2486.4,15.622,285307.2201,18679.66354,23.5476,24.3911,23.1161,...,18.280,10.238,2.17,2.48,2.08,95.3,6753.0,12646.0,80.7315,NaN


We will follow the steps below to clean data and make it ready for feature selection process.

1. Remove the variables with missing observations
2. Add lags of the variables as additional features
3. Test stationarity of time series
4. Standardize the dataset

In [4]:
#remove columns with missing observations
missing_colnames=[]
for i in bigmacrodata.drop(['Date','Regime'],axis=1):
    observations=len(bigmacrodata)-bigmacrodata[i].count()
    if (observations>0):
        print(i+':'+str(observations))
        missing_colnames.append(i)

print(len(missing_colnames))
bigmacrodata=bigmacrodata.drop(labels=missing_colnames, axis=1)

#rows with missing values
bigmacrodata=bigmacrodata.dropna(axis=0)

bigmacrodata.shape

CMRMTSPLx:1
HWI:1
HWIURATIO:1
PERMIT:12
PERMITNE:12
PERMITMW:12
PERMITS:12
PERMITW:12
ACOGNO:398
ANDENOx:109
BUSINVx:1
ISRATIOx:1
NONREVSL:1
CONSPI:1
S&P div yield:2
S&P PE ratio:1
CP3Mx:1
COMPAPFFx:1
TWEXAFEGSMTHx:168
UMCSENTx:154
DTCOLNVHFNM:1
DTCTHFNM:1
VIXCLSx:42
23


(788, 105)

In [5]:
# Add lags
for col in bigmacrodata.drop(['Date', 'Regime'], axis=1):
    for n in [3,6,9,12,18]:
        bigmacrodata['{} {}M lag'.format(col, n)] = bigmacrodata[col].shift(n).ffill().values

# 1 month ahead prediction
bigmacrodata["Regime"]=bigmacrodata["Regime"].shift(-1)

bigmacrodata=bigmacrodata.dropna(axis=0)

In [6]:
bigmacrodata.shape

(769, 620)

In [7]:
#check stationarity
threshold=0.01 #significance level
for column in bigmacrodata.drop(['Date','Regime'], axis=1):
    result=adfuller(bigmacrodata[column])
    if result[1]>threshold:
        # use first difference to make series statinary
        bigmacrodata[column]=bigmacrodata[column].diff()
bigmacrodata=bigmacrodata.dropna(axis=0)

In [8]:
threshold=0.01 #significance level
for column in bigmacrodata.drop(['Date','Regime'], axis=1):
    result=adfuller(bigmacrodata[column])
    if result[1]>threshold:
        # use first difference again to make series statinary
        bigmacrodata[column]=bigmacrodata[column].diff()
bigmacrodata=bigmacrodata.dropna(axis=0)

In [9]:
# Standardize
features=bigmacrodata.drop(['Date','Regime'],axis=1)
col_names=features.columns

scaler=StandardScaler()
scaler.fit(features)
standardized_features=scaler.transform(features)
standardized_features.shape
df=pd.DataFrame(data=standardized_features,columns=col_names)
df.insert(loc=0,column="Date", value=bigmacrodata['Date'].values)
df.insert(loc=1,column='Regime', value=bigmacrodata['Regime'].values)
df.head()
df.shape

(767, 620)

In [11]:
# writing cleaned data to new csv

df.to_csv("data/Dataset_Cleaned.csv", index=False)